In [ ]:
# !pip install plotly matplotlib pandas requests
# !git clone https://github.com/HuobiRDCenter/huobi_Python
# !cd huobi_Python && python setup.py install

In [ ]:
import datetime
import requests
import pandas as pd
import random
from utils import *
from strategies import *
from matplotlib import pyplot as plt

In [ ]:
symbol_list = get_symbol_list()
sb = symbol_list[random.randint(0, len(symbol_list))]
for sb in symbol_list:
    a=get_price(sb, "30min", 2000)
    a=get_rate(a, "close")
    a["cr+1"] = (a["close"].shift(-1) - a["close"]) / a["close"]
    a["cr+2"] = (a["close"].shift(-2) - a["close"]) / a["close"]
    a=get_rate(a, "volume")
    a=custom_data_analyze_func(a, rolling_days=3)
    standard = a.describe()["vol_mean3_cr"]["75%"]
    c=a.loc[(a["vol_mean3_cr"]>standard) & (a["close_rate"]>0)]
    try:
        if datetime.datetime.strptime("2021-04-22 17:00:00", "%Y-%m-%d %H:%M:%S") == c.index[-1]:
            print(sb)
    except:
        continue
print("done")

In [ ]:
# symbol_list = get_symbol_list()
# sb = symbol_list[random.randint(0, len(symbol_list))]
sb = "mtausdt"
a=get_price(sb, "30min", 2000)
print(sb)
a=get_rate(a, "close")
a["cr+1"] = (a["close"].shift(-1) - a["close"]) / a["close"]
a["cr+2"] = (a["close"].shift(-2) - a["close"]) / a["close"]
a=get_rate(a, "volume")
a=custom_data_analyze_func(a, rolling_days=3)
standard = a.describe()["vol_mean3_cr"]["75%"]
c=a.loc[(a["vol_mean3_cr"]>standard) & (a["close_rate"]>0)]
print(c.index[-1], c.iloc[-1]["close"])
display(c[["cr+1","cr+2","close_rate"]].describe())
# c[["cr+1"]].plot(kind="hist")
aa=get_price(sb, "1min", 2000)
aa=get_mean_line(aa)
bb = aa[c.index[-1]-datetime.timedelta(hours=0.5) : c.index[-1]]
draw_candle(bb)
cc = aa[c.index[-1]:c.index[-1]+datetime.timedelta(hours=0.5)]
draw_candle(cc)